# TODO make pip model that takes bigquery query and question and will produce answer from results in query

In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [1]:
%load_ext google.colab.data_table

In [4]:
import pandas as pd

project_id = 'stackoverflow-qa'
sample_count = 2000
df = pd.io.gbq.read_gbq('''
SELECT q.title, q.body AS `q_body`, a.body AS `a_body`, a.score, (case when q.accepted_answer_id = a.id then 1 else 0 end) AS `acc_a`
FROM `bigquery-public-data.stackoverflow.posts_answers` AS a
INNER JOIN `bigquery-public-data.stackoverflow.posts_questions` AS q ON a.parent_id = q.id
WHERE a.score > 0
LIMIT 100
''', project_id = project_id, dialect= 'standard')

df.head()

,title,q_body,a_body,score,acc_a
0,List Mutation on Python,<p>I tried mutating a list by swapping a commo...,"<p>Although in Python, the right-hand side is ...",12,0
1,Simple example for why Same Origin Policy is n...,<p>I've read about <code>Same Origin Policy</c...,"<pre><code>&lt;iframe id=""bank"" src=""https://y...",36,1
2,Adding an image to the JFrame,"<p>I want to add an Image to the JFrame, the <...",<p>Visibility of <code>JFrame jfrm</code> is l...,4,1
3,How to parse nested json arrays in a single ob...,<p>I have a json which has a number of nested ...,<p>So here is my solution:</p>\n\n<pre><code>J...,6,0
4,List Mutation on Python,<p>I tried mutating a list by swapping a commo...,<p>The problem is that the two are not equival...,7,1


# CODE FROM:
https://www.kaggle.com/officialshivanandroy/question-answering-with-arxiv-papers-at-scale

In [5]:
# installing haystack

! pip install git+https://github.com/deepset-ai/haystack.git

ack==0.3.0) (4.41.1)
     |████████████████████████████████| 748.8MB 21kB/s 
     |████████████████████████████████| 778kB 34.9MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 8.7MB 35.2MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 3.9MB 43.3MB/s 
     |████████████████████████████████| 81kB 7.1MB/s 
     |████████████████████████████████| 225kB 41.1MB/s 
     |████████████████████████████████| 4.9MB 29.6MB/s 
     |████████████████████████████████| 163kB 33.9MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 133kB 44.4MB/s 
     |████████████████████████████████| 153kB 39.6MB/s 
     |████████████████████████████████| 163kB 34.7MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 890kB 29.7MB/s 
     |████████████████████████████████| 3.0MB 37.6MB/s 
     |███████████████████████████████

In [6]:
# importing necessary dependencies

from haystack import Finder
from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

In [7]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2
 
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [8]:
from haystack.database.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

In [9]:
# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(df[['title', 'a_body']].rename(columns={'title':'name','a_body':'text'}).to_dict(orient='records'))

In [10]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [11]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True, context_window_size=500)

INFO:farm.utils:device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
INFO:farm.infer:Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...



	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.



INFO:farm.utils:device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
INFO:farm.infer:Got ya 1 parallel workers to do inference ...
INFO:farm.infer: 0 
INFO:farm.infer:/w\
INFO:farm.infer:/'\
INFO:farm.infer:


In [12]:
finder = Finder(reader, retriever)

In [13]:
prediction = finder.get_answers(question="What do we know about symbiotic stars", top_k_retriever=10, top_k_reader=2)
result = print_answers(prediction, details="minimal")

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.06 Batches/s][   {   'answer': 'you might already know',
        'context': 'Username=postgres;Password=123456;database=testdb" );\n'
                   '...\n'
                   '</code></pre>\n'
                   '\n'
                   '<p>(<code>MyType</code> must have a public constructor not '
                   'taking any arguments for '
                   '<code>NpgsqlConnection.MapCompositeGlobally&lt;&gt;()</code> '
                   'to work with it. But you might already know that.)</p>\n'
                   '\n'
                   '<p>Additionally you must set the Property '
                   '<code>SpecificType</code> of the '
                   "<code>NpgsqlParameter</code> to your C# type. That's what "
                   "you seem to have actually missed. (It doesn't matter where "
                   "you do that (as long as it's befor"},
    {   'answer': 'will not work',
        'context': 'cate the stri

In [14]:
df.head()

,title,q_body,a_body,score,acc_a
0,List Mutation on Python,<p>I tried mutating a list by swapping a common element between the list and...,"<p>Although in Python, the right-hand side is evaluated <em>first</em> when ...",12,0
1,Simple example for why Same Origin Policy is needed,"<p>I've read about <code>Same Origin Policy</code>, but for a better underst...","<pre><code>&lt;iframe id=""bank"" src=""https://yourbank.com""&gt;&lt;/iframe&gt...",36,1
2,Adding an image to the JFrame,"<p>I want to add an Image to the JFrame, the <code>add( )</code> method is g...",<p>Visibility of <code>JFrame jfrm</code> is limited by constructor of <code...,4,1
3,How to parse nested json arrays in a single object in android,<p>I have a json which has a number of nested JSONARRAY. here is the snippet...,<p>So here is my solution:</p>\n\n<pre><code>JSONObject first = new JSONObje...,6,0
4,List Mutation on Python,<p>I tried mutating a list by swapping a common element between the list and...,<p>The problem is that the two are not equivalent. The first example is akin...,7,1


In [ ]:
prediction = finder.get_answers(question="How to parse nested json arrays", top_k_retriever=10, top_k_reader=2)
result = print_answers(prediction, details="minimal")

# SCRATCH WORK